# Recommendation System - Project 2

### [0. Imports](#0)

### [1. Data Exploration](#1)

### [2. Model Building](#2)

### [3. Cross Validation](#3)

### [4. Run Model](#4)

### [5. Produce Predictions](#5)

## 0. Imports <a class="anchor" id="0"></a>

In [13]:
# Useful starting lines
%matplotlib inline

import numpy as np
import scipy
import scipy.io
import scipy.sparse as sp
import matplotlib.pyplot as plt
from helpers import *
from functions import *
import warnings

warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 1. Data Exploration <a class="anchor" id="1"></a>

**In this section we analyse the collected data:**

We follow on with the data split. As the matrices we will be working with are sparse, we will use the `sp.lil_matrix` representation:<br>
    
An array (self.rows) of rows, each of which is a sorted list of column indices of non-zero elements The corresponding nonzero values are stored in similar fashion in self.data.
    

The following function will be used to split the data:

In [14]:
def split_data(ratings, p_test=0.1, seed=516):
    """
    split the ratings to training data and test data.
    ***INPUT***
    :param min_num_ratings: all users and items we keep must have at least min_num_ratings
                            per user and per item. 
    :param p_test: probability for test split
    :param seed: seed to use in random split
    ***OUTPUT***
    :return train: data for training
    :return test: data for test
    """
    np.random.seed(seed) 

    # initialize matrix
    train = sp.lil_matrix(ratings.shape)
    test  = sp.lil_matrix(ratings.shape)
    
    nz_items, nz_users = ratings.nonzero()
    
    # split the data
    for idx in zip(nz_items, nz_users):
        if np.random.rand() > p_test:
            train[idx] = ratings[idx]
        else:
            test[idx] = ratings[idx]
            
    return train, test

## 2. Model Building<a class="anchor" id="2"></a>

**Matrix Initialization:**

In [15]:
def init_MF(train, num_features):
    """init the parameter for matrix factorization.
    ***INPUT***
    :param train: data for train
    :param num_features: nummber of features for matrix factorization
    ***OUTPUT***
    :return user_features: user's matrix
    :return item_features: item's matrix
    """
    # get shapes    
    num_item, num_user = train.get_shape()

    # random initialization
    user_features = np.random.rand(num_features, num_user)
    item_features = np.random.rand(num_features, num_item)

    # start by item features.
    item_nnz = train.getnnz(axis=1)
    item_sum = train.sum(axis=1)

    for ind in range(num_item):
        item_features[0, ind] = item_sum[ind, 0] / item_nnz[ind]
        
    return user_features, item_features

In [16]:
def initialize_parameters(train, num_features, seed):
    """
    Initializes parameters tu use in ALS
    ***INPUT***
    :param train: data for train
    :param num_features: number of features for Matrix factorization
    ***OUTPUT***
    :param seed: seed for random generator
    :return user_features: user's matrix
    :return item_features: item's matrix
    :return bias_user: user's bias vector
    :return bias_item: item's bias vector
    """
    # set seed
    np.random.seed(seed)  #np.random.seed(84)
    
    # initialize biases
    num_items, num_users = train.shape
    bias_item = np.zeros(num_items) #np.random.rand(num_items) - 0.5
    bias_user = np.zeros(num_users) #np.random.rand(num_users) - 0.5

    # init ALS
    user_features, item_features = init_MF(train, num_features)
    
    return user_features, item_features, bias_item, bias_user

**Matrix Update**:

- Update `user features`:

In [17]:
def update_user_feature(train, item_features, lambda_user,
                        nnz_items_per_user, nz_user_itemindices, 
                        bias_item, bias_user, lambda_bias_user):   
    """
    Update user feature matrix, for fixed item matrix
    ***INPUT***
    :param train: data for train
    :param item_features: item's matrix
    :param lambda_user: regularization parameter 
    :param nnz_items_per_user: non zero items per each user
    :param nz_user_itemindices: indices of non zero users per item
    :param bias_item: bias per item
    :param bias_user: bias per user
    :param lambda_bias_user: regularization parameter for user bias
    ***OUTPUT***
    :return user_features_new: updated user's matrix
    :return bias_user_new: updated bias user
    """
    
    num_users = train.shape[1]
    bias_user_new = np.zeros(num_users)
    k = item_features.shape[0]
    lambda_I = lambda_user * sp.eye(k)
    user_features_new = np.zeros((k, num_users))
    
    for user in range(num_users):
        # extract the columns corresponding to the prediction for the given item
        M = item_features[:, nz_user_itemindices[user]]
        
        # get non zero values
        user_nz = train[nz_user_itemindices[user] , user].toarray().squeeze()
        bias_item_nz = bias_item[nz_user_itemindices[user]]
        
        # update column of user features
        user_features_new = update_column_user(user, user_nz, nnz_items_per_user, M,
                                                        lambda_I, user_features_new)

        # update user bias
        bias_user_new = update_user_bias(user, user_nz, bias_item_nz, user_features_new,
                                         nnz_items_per_user, M, bias_user_new, lambda_bias_user)
    
    return user_features_new, bias_user_new

- Update `item features`:

In [18]:
def update_item_feature(train, user_features, lambda_item,
                        nnz_users_per_item, nz_item_userindices, 
                        bias_item, bias_user, lambda_bias_item):
    """
    Update item feature matrix.
    ***INPUT***
    :param train: data for train
    :param user_features: item's matrix
    :param lambda_item: regularization parameter 
    :param nnz_users_per_item: non zero users per each item
    :param nz_item_userindices: indices of non zero items per user
    :param bias_item: bias per item
    :param bias_user: bias per user
    :param lambda_bias_item: regularization parameter for item bias
    ***OUTPUT***
    :return user_features_new: updated user's matrix
    :return bias_user_new: updated bias user
    """
    
    num_items = train.shape[0]
    bias_item_new = np.zeros(num_items)
    k = user_features.shape[0]  
    lambda_I = lambda_item * sp.eye(k)
    item_features_new = np.zeros((k, num_items))
    
    for item in range(num_items):
        # extract the columns corresponding to the prediction for the given user
        M = user_features[:, nz_item_userindices[item]]
        
        # get non zero values
        item_nz = train[item, nz_item_userindices[item]].toarray().squeeze()
        bias_user_nz = bias_user[nz_item_userindices[item]]

        # update column of item features
        item_features_new = update_column_item(item, item_nz, nnz_users_per_item, M,
                                                        lambda_I, item_features_new)
        # update item bias
        bias_item_new = update_item_bias(item, item_nz, bias_user_nz, item_features_new,
                                         nnz_users_per_item, M, bias_item_new, lambda_bias_item)

    return item_features_new, bias_item_new

**Matrix Factorization with ALS**:

We define the following auxiliar functions:

**The following function runs the `ALS`:**

In [19]:
def run_ALS(train, user_features, item_features, bias_user, bias_item, lambda_user,lambda_item,
            lambda_bias_item, lambda_bias_user, nnz_items_per_user, nz_user_itemindices,
            nnz_users_per_item, nz_item_userindices, nz_train, stop_criterion):
    """
    Performs Alternating Least Squares
    ***INPUT***
    :param train: data used for training
    :param user_features: user's matrix
    :param item_features: item's matrix
    :param bias_user: array with bias for users
    :param bias_item: array with bias for items
    :param lambda_user: regularization parameter for users
    :param lambda_item: regularization parameter for items
    :param lambda_bias_item: regularization parameter for items' biases
    :param lambda_bias_user: regularization parameter for users' biases
    :param nnz_items_per_user: # of non zero items per user
    :param nz_user_itemindices: indices of non zero users per item
    :param nnz_users_per_item: # of non zero users per item
    :param nz_item_userindices: indices of non zero items per user
    :param nz_train: tuple of non zero (row, col)
    :param stop_criterion: minimum difference between two consecutive iterations
    ***OUTPUT***
    :return user_features: user's matrix
    :return item_features: item's matrix
    :return bias_user: user's bias vector
    :return bias_item: item's bias vector
    """
    # define parameters
    change = 1
    error_list = [0]
    
    print("\nstart the ALS algorithm with biases...")
    while change > stop_criterion:
        
        # update user feature & item feature
        user_features, bias_user = update_user_feature(
            train, item_features, lambda_user,
            nnz_items_per_user, nz_user_itemindices, 
            bias_item, bias_user, lambda_bias_user)
        
        item_features, bias_item = update_item_feature(
            train, user_features, lambda_item,
            nnz_users_per_item, nz_item_userindices, 
            bias_item, bias_user, lambda_bias_item)

        error = compute_error_ALS(train, user_features, item_features, 
                                  bias_user, bias_item, nz_train)
        print("RMSE on training set: {}.".format(error))
        error_list.append(error)
        change = np.fabs(error_list[-1] - error_list[-2])
    
    return user_features, item_features, bias_user, bias_item

In [20]:
def ALS_bias(train, test, num_features,lambda_user, lambda_item, lambda_bias_user,
             lambda_bias_item, stop_criterion, seed):
    """
    Alternating Least Squares (ALS) algorithm with biases.
    ***INPUT***
    :param train: data for train
    :param test: data for test
    :param num_features: number of features for Matrix factorization
    :param lambda_user: regularization parameter for user
    :param lambda_item: regularization parameter for item 
    :param lambda_bias_user: regularization parameter for user bias
    :param lambda_bias_item: regularization parameter for item bias
    :param stop_criterion: minimum difference between two consecutive iterations
    :param seed: seed for random generator
    ***OUTPUT***
    :return user_features: user's matrix
    :return item_features: item's matrix
    :return bias_user: user's bias vector
    :return bias_item: item's bias vector
    :return rmse: rmse error
    """
    
    #Initialize parameters:
    user_features, item_features, bias_item, bias_user = initialize_parameters(train, num_features,seed)
    
    # get non zero values
    nz_row ,nz_col, nz_train, nnz_users_per_item, nz_item_userindices, nnz_items_per_user, nz_user_itemindices = get_non_zero_values(train)
    
    # run ALS
    user_features, item_features, bias_user, bias_item = run_ALS(
        train,
        user_features, item_features,
        bias_user, bias_item,
        lambda_user, lambda_item, 
        lambda_bias_item, lambda_bias_user,
        nnz_items_per_user, nz_user_itemindices, 
        nnz_users_per_item, nz_item_userindices,
        nz_train,
        stop_criterion)

    # evaluate the test error
    rmse = evaluate(test, user_features, item_features, bias_user, bias_item)
    
    return user_features, item_features, bias_user, bias_item, rmse

## 3. Cross Validation<a class="anchor" id="3"></a>

Define plotting function:

In [21]:
def CV_boxplot(errors, n_features):
    """
    Plots a box plot for CV tested with different number of features
    ***INPUT***
    :param errors: errors for different number of features tested
    :param n_features: labels for plot
    """
    _ = plt.boxplot(errors, labels = n_features, meanline=True, autorange= True)
    plt.title("Error for different number of features")
    plt.xlabel("Number of features")
    plt.ylabel("Test error")

`Cross Validation` will be performed in order to minimize the overfitting error:

In [22]:
def CV(ratings, ALS_parameters, k_folds, seeds):
    """
    Performs CV over the number of specified folds.
    Plots if True
    ***INPUT***
    :param ratings: values to consider for data split
    :param ALS-parameters: list that contain all the parameters for ALS
    :param k_folds: number of folds to test
    :param seeds: list of seeds to apply for consecutive folds
    ***OUTPUT***
    :return user_features_avg: matrix of users avereged over folds
    :return item_features_avg: matrix of items avereged over folds
    :return bias_user_avg: array of biases per user avereged over folds
    :return bias_item_avg: array of biases per item avereged over folds
    :return rmse_avg: CV mean error avereged over folds
    """
    
    # define list to append folds values
    user_features_folds = []; item_features_folds = []; 
    bias_user_folds= []; bias_item_folds = []; 
    error_folds=[]
    
    for fold_id in range(k_folds):
        train_k, test_k = split_data(ratings, 0.1, seeds[fold_id])

        user_features, item_features, bias_user, bias_item, rmse = ALS_bias(train_k, test_k, ALS_parameters[0],
                                                                           ALS_parameters[1], ALS_parameters[2], 
                                                                           ALS_parameters[3], ALS_parameters[4],
                                                                           ALS_parameters[5], seed=seeds[fold_id])

        print("Error for fold ,",fold_id+1, 'is: ', rmse)

        # append fold values
        user_features_folds.append(user_features); 
        item_features_folds.append(item_features)
        bias_user_folds.append(bias_user)
        bias_item_folds.append(bias_item)
        error_folds.append(rmse)

        #average parameters
        user_features_avg, item_features_avg, bias_user_avg, bias_item_avg, rmse_avg =  average_parameters(user_features_folds,
                                                                             item_features_folds, bias_user_folds,
                                                                                 bias_item_folds, error_folds)

    return user_features_avg, item_features_avg, bias_user_avg, bias_item_avg, rmse_avg, error_folds
    
def average_parameters(user_features_folds, item_features_folds,bias_user_folds,
                       bias_item_folds,error_folds):
    """
    Performs average on matrices obtained for each fold
    ***INPUT***
    :param user_features_folds: array with user's matrix for each fold
    :param item_features_folds: array with item's matrix for each fold
    :param bias_user_folds: array with user biases array for each fold
    :param bias_item_folds: array with item biases array for each fold
    :param error_folds: array with error for each fold
    ***OUTPUT***
    :return user_features: matrix of users avereged over folds
    :return item_features: matrix of items avereged over folds
    :return bias_user: array of biases per user avereged over folds
    :return bias_item: array of biases per item avereged over folds
    :return rmse: CV mean error avereged over folds
    """
    # average arrays and return final matrices:
    user_features = np.mean(user_features_folds, axis = 0)
    item_features = np.mean(item_features_folds, axis = 0)
    bias_user = np.mean(bias_user_folds, axis = 0)
    bias_item = np.mean(bias_item_folds, axis = 0)
    rmse = np.mean(error_folds, axis = 0)
    
    return user_features, item_features, bias_user, bias_item, rmse
  

def ALS_cross_validation(ratings, k_folds, ALS_parameters, plot_n_features = False, n_features = []):
    """
    Performs Cross Validation.
    ***INPUT***
    :param ratings: values to consider for data split
    :param k_folds: number of folds to test
    :param plot_n_features: boolean to decide if CV will be ran for different number of features
    :param n_features: vector with features to test (if plot_n_features == True)
    :param ALS_parameters: list that contains the parameters to apply in ALS: [0]- # of features
                            [1]- lambda_user
                            [2]- lambda_item
                            [3]- lambda_bias
                            [4]- lambda_bias_item
                            [5]- stop_criterion
    ***OUTPUT***
    :return user_features: matrix of users avereged over folds (corresponding to min rmse)
    :return item_features: matrix of items avereged over folds (corresponding to min rmse)
    :return bias_user: array of biases per user avereged over folds (corresponding to min rmse)
    :return bias_item: array of biases per item avereged over folds (corresponding to min rmse)
    :return rmse: CV mean error avereged over folds (corresponding to min rmse)
    """
    
    # seeds to use in data spliting
    seeds = np.random.randint(1, 500, size = k_folds)
    print(seeds)
    
    # perform ALS for different number of features
    if (plot_n_features == True):
        errors = []
        rmse_min = 10  #initialize with high value
        for idx in range(len(n_features)):
            ALS_parameters[0] = n_features[idx]
            user_features_idx, item_features_idx, bias_user_idx, \
            bias_item_idx, rmse_idx, error_folds = CV(ratings, ALS_parameters, k_folds, seeds)
            if rmse_idx < rmse_min: 
                user_features = user_features_idx
                item_features =item_features_idx
                bias_user= bias_user_idx
                bias_item = bias_item_idx
                rmse = rmse_idx
                rmse_min = rmse_idx
            errors.append(error_folds)  

        #plot results
        CV_boxplot(errors, n_features)
    
    # performs ALS for fixed number of features
    else: 
        user_features, item_features, bias_user, \
        bias_item, rmse, error_folds =  CV(ratings, ALS_parameters, k_folds, seeds)
        
    return  user_features, item_features, bias_user, bias_item, rmse

## 4. Apply Model (RUN)<a class="anchor" id="4"></a>

** Data loading: **

In [23]:
path_dataset = "./Data/data_train.csv"
ratings = load_data(path_dataset)

number of items: 10000, number of users: 1000


In [24]:
# parameters to use:
"""
[0]- number of features
[1]- lambda_user
[2]- lambda_item
[3]- lambda_bias
[4]- lambda_bias_item
[5]- stop_criterion
"""

def run(option , parameters, folds=4):
    """
    Function to run the model. Can run either one of the three options:
        1: ALS
        2: ALS with cross validation
        3: ALS with cross-validation and plot for different features
    ***INPUT***
    :param option: 
    :param parameters: 
    :param folds: 
    ***OUTPUT***
    """
    # RUN ALS
    if option == 1:
        # same as running cross validation with 1 fold
        user_features, item_features, bias_user, bias_item, rmse = ALS_cross_validation(ratings, 1, parameters) 
    
    # RUN ALS WITH CV
    elif option == 2:
        user_features, item_features, bias_user, bias_item, rmse = ALS_cross_validation(ratings, folds, parameters)
        
    # RUN ALS WITH CV AND PLOT FOR DIFFERENT NUMBER OF FEATURES
    else:
        n_features = [20, 25, 30, 35, 40]
        user_features, item_features, bias_user, bias_item, rmse = ALS_cross_validation(ratings, folds, parameters,
                                                                                        plot_n_features=True,
                                                                                        n_features=n_features)
        
    return user_features, item_features, bias_user, bias_item, rmse
        

**Choose which version to run:**
- 1: `ALS`
- 2: `ALS with Cross Validation`
- 3: `ALS with Cross Validation and plot for different features`

In [25]:
parameters = [25, 0.1, 0.1, 0.1, 0.1, 0.5e-5]
user_features, item_features, bias_user, bias_item, rmse = run(option=1 , parameters=parameters)

[396]

start the ALS algorithm with biases...
RMSE on training set: 0.9916097125655676.
RMSE on training set: 0.9799149946047435.
RMSE on training set: 0.9648702780440435.
RMSE on training set: 0.9485932024896112.
RMSE on training set: 0.9368582478882101.
RMSE on training set: 0.9288207905514376.
RMSE on training set: 0.9235377200888822.
RMSE on training set: 0.9200028253769351.
RMSE on training set: 0.9175849139800518.
RMSE on training set: 0.9158861020482506.
RMSE on training set: 0.9146559751124483.
RMSE on training set: 0.9137372585789946.
RMSE on training set: 0.91303075036754.
RMSE on training set: 0.9124731260070268.
RMSE on training set: 0.91202315488226.
RMSE on training set: 0.91165329994521.
RMSE on training set: 0.9113446439811186.
RMSE on training set: 0.9110838145241826.
RMSE on training set: 0.9108610958495913.
RMSE on training set: 0.9106692479544896.
RMSE on training set: 0.9105027516500401.
RMSE on training set: 0.9103573148415639.
RMSE on training set: 0.910229541988

## 5. Produce Predictions<a class="anchor" id="5"></a>

Produce `submission`:

In [26]:
SUBMISSION_SAMPLES_PATH = "./Data/sample_submission.csv"
samples_submission      = samples_csv_submission(SUBMISSION_SAMPLES_PATH)

create_csv_submission(samples_submission, item_features, user_features, bias_item, bias_user, 'submission.csv')